In [ ]:
# standard libraries
import os
from datetime import date, timezone
import openpyxl

# imported libraries
import requests
from dotenv import load_dotenv
import pandas as pd
import xlsxwriter

In [ ]:
# specify date range
commencement = "2022-08-03"
today = str(date.today())

In [ ]:
output = os.path.join(os.getcwd(), "File Outputs", "LendSaaS")

In [ ]:
# load api key & dictate base url

load_dotenv()
api_key = os.getenv("LS_APIKEY")
base_url = "https://b2bfinancing.lendtech.io/backend/api/partners"

In [ ]:
# get underwriting info within a given date range
def get_underwriting_info(start, end):
    url = f"{base_url}/underwriting-info?submittedMinDate={start}&submittedMaxDate={end}"
    uw = requests.get(
        url,
        headers={"x-api-key": api_key}
    )
    return uw.json()

In [ ]:
# get lead API call based on lead ID in each underwriting submission
def get_lead(obj, id):
    url = f"{base_url}/lead/{id}"
    lead = requests.get(
        url,
        headers={"x-api-key": api_key}
    )
    obj["lead"] = lead.json()

In [ ]:
# get offers API call based on lead ID in each underwriting submission
def get_offers(obj, id):
    url = f"{base_url}/offers?leadId={id}"
    offers = requests.get(
        url,
        headers={"x-api-key": api_key}
    )
    obj["offers"] = offers.json()

In [ ]:
# get positions API call based on lead ID in each underwriting submission
def get_positions(obj, id):
    url = f"{base_url}/positions?leadId={id}"
    positions = requests.get(
        url,
        headers={"x-api-key": api_key}
    )
    obj["positions"] = positions.json()

In [ ]:
# get uw-status history API call based on lead ID in each underwriting submission
def get_status_history(obj, id):
    url = f"{base_url}/uw-status-history?leadId={id}"
    status_history = requests.get(
        url,
        headers={"x-api-key": api_key}
    )
    obj["statusList"] = status_history.json()

In [ ]:
# account monitoring (asset management style data)
def get_account_snapshots():
    url = f"{base_url}/snapshots/account-monitoring"
    payload = {
        "amStatusId": "1",
        # 1 = Performing
        # 2 = Pending
        "includeClosedDeals": True,
        "includeWriteoffDeals": True
    }
    
    am = requests.post(
        url,
        headers={"x-api-key": api_key},
        json=payload
    )

    return am.json()


In [ ]:
# funding (funding summary)
def get_funding_snapshots():
    url = f"{base_url}/snapshots/funding"
    payload = {
        "sdate":commencement,
        "edate":today
    }
    
    fundings = requests.post(
        url,
        headers={"x-api-key": api_key},
        json=payload
    )

    return fundings.json()

In [ ]:
# get transactions from monitored accounts
def get_transactions(start, end, obj, id):
    url = f"{base_url}/transactions-details?datePostedMin={start}&datePostedMax={end}&leadOd={id}"
    transactions = requests.get(
        url,
        headers={"x-api-key": api_key}
    )
    obj["transactions"] = transactions.json()

In [ ]:
# lead level parser for nested info
def parse(obj):

    # search obj
    status = obj["currentStatus"]
    lead = obj["lead"]
    iso = lead["iso"] or {"isoName": "", "baseFactor": 0, "upfrontComm": 0, "residualComm": 0, "backendComm": 0}

    # status
    obj["ISO Manager"] = status["username"]

    # iso
    obj["ISO Name"] = iso["isoName"]
    obj["Base Factor"] = iso["baseFactor"]
    obj["Upfront Comm"] = iso["upfrontComm"]
    obj["Residual Comm"] = iso["residualComm"]
    obj["Backend Comm"] = iso["backendComm"]

    # lead
    obj["Legal Name"] = lead["entityName"]
    obj["DBA"] = lead["entityDba"]
    obj["Address"] = lead["entityAddress1"]
    obj["City"] = lead["entityCity"]
    obj["State"] = lead["entityState"]
    obj["Amount Requested"] = lead["amountRequested"]
    obj["Loan Purpose"] = lead["loanPurpose"]

    return obj


In [ ]:
# assign names to the numeric underwriting statuses
uw_statuses = {
    1: "Pre Underwriting",
    2: "Initial Underwriting",
    3: "In Queue",
    4: "On Hold",
    5: "Offers Sent",
    6: "Offer Selected",
    7: "Contracts Out",
    8: "Contracts In",
    9: "Relook",
    10: "Declined",
    11: "Funded",
    12: "Pre Funding",
    13: "Ready for Pre Underwriting",
    14: "New Lead",
    15: "App out",
    16: "Contract on Hold",
    20: "Docs Rqd For Offer",
    21: "PIPE DRIVE",
    22: "Inactive",
    23: "Withdrawn"
}

In [ ]:
# pull underwriting info
uw_info = get_underwriting_info(commencement, today)

In [ ]:
# get lead info and parse data
for lead in uw_info:
    get_lead(lead, lead["leadId"])
    get_offers(lead, lead["leadId"])
    get_positions(lead, lead["leadId"])
    get_status_history(lead, lead["leadId"])
    parse(lead)

In [ ]:
offers = []
statuses = []
monthly_bank_totals = []
positions = []

for lead in uw_info:
    offers.extend(lead["offers"])
    statuses.extend(lead["statusList"])
    positions.extend(lead["positions"])
    for month in lead["monthlyBankingTotals"]:
        month["leadId"] = lead["leadId"]
    monthly_bank_totals.extend(lead["monthlyBankingTotals"])
    

In [ ]:
servicing = get_account_snapshots()
fundings = get_funding_snapshots()

In [ ]:
deals = servicing["deals"]
transactions = []

for deal in deals:
    get_transactions(commencement, today, deal, deal["leadId"])
    transactions.extend(deal["transactions"])

In [ ]:
# deal desk

# synds = []

# for deal in deals:
#     synds.extend(deal["syndications"])

In [ ]:
uw = pd.DataFrame.from_dict(uw_info)
of = pd.DataFrame.from_dict(offers)
st = pd.DataFrame.from_dict(statuses)
mb = pd.DataFrame.from_dict(monthly_bank_totals)
po = pd.DataFrame.from_dict(positions)
am = pd.DataFrame.from_dict(deals)
tx = pd.DataFrame.from_dict(transactions)
fd = pd.DataFrame.from_dict(fundings)
# dd = pd.DataFrame.from_dict(synds)

In [ ]:
for i, row in uw.iterrows():   
    uw.loc[i, "uwStatus"] = uw_statuses[row.uwStatus]

In [ ]:
def change_to_utc(time):
    return time.astimezone(timezone.utc).date()

In [ ]:
# convert columns to numbers or dates if possible
dfs = [uw, of, st, po, mb, am, tx, fd] #dd

for df in dfs:
    for col in df.columns:
        try:
            df[col] = df[col].apply(pd.to_numeric)
        except:
            if col.__contains__("Date"):
                try:
                    df[col] = df[col].apply(pd.to_datetime)
                except:
                    continue
                try:
                    df[col] = df[col].apply(change_to_utc)
                except:
                    continue
            continue

In [ ]:
with pd.ExcelWriter(os.path.join(output, f"B2B Summary {today}.xlsx"), engine="xlsxwriter", date_format="MM/DD/YYYY", datetime_format="MM/DD/YYYY") as writer:
    
    # formats
    wb  = writer.book
    financial = wb.add_format({'num_format': '_(* #,##0.00_);_(* (#,##0.00);_(* "-"??_);_(@_)'})
    percent = wb.add_format({'num_format': '0.0000%'})
        
    # write frames to tabs
    uw.to_excel(writer, sheet_name="UW Info", index=False)
    of.to_excel(writer, sheet_name="Offers", index=False)
    st.to_excel(writer, sheet_name="Status History", index=False)
    po.to_excel(writer, sheet_name="Positions", index=False)
    mb.to_excel(writer, sheet_name="Banking Totals", index=False)
    am.to_excel(writer, sheet_name="Accounts", index=False)
    tx.to_excel(writer, sheet_name="Transactions", index=False)
    fd.to_excel(writer, sheet_name="Fundings", index=False)
    # dd.to_excel(writer, sheet_name="Deal Desk", index=False)
    
    # FORMAT SHEETS
    ## name sheets
    uw_tab = writer.sheets["UW Info"]
    of_tab = writer.sheets["Offers"]
    st_tab = writer.sheets["Status History"]
    po_tab = writer.sheets["Positions"]
    mb_tab = writer.sheets["Banking Totals"]
    am_tab = writer.sheets["Accounts"]
    tx_tab = writer.sheets["Transactions"]
    fd_tab = writer.sheets["Fundings"]
    # dd_tab = writer.sheets["Deal Desk"]
    
    worksheets = {uw_tab:uw, of_tab:of, st_tab:st, po_tab:po, mb_tab:mb, am_tab:am, tx_tab:tx, fd_tab:fd} # dd_tab:dd, 
    
    for sheet in worksheets.keys():
        (max_row, max_col) = worksheets[sheet].shape
        sheet.autofilter(0, 0, max_row, max_col - 1)
        